<a href="https://colab.research.google.com/github/AdamClarkStandke/ReinforcementLearning/blob/main/forward_weights_90000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install ptan
!pip install -q pybullet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import gym
import ptan
import time
import torch
import microtaur, ddpg



net = None
net = ddpg.DDPGActor(28, 4)
print(net)
net.load_state_dict(torch.load('best_+0.087_90000.dat', map_location=lambda storage, loc: storage))


DDPGActor(
  (net): Sequential(
    (0): Linear(in_features=28, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=4, bias=True)
    (3): Tanh()
  )
)


<All keys matched successfully>

In [78]:
x=net.state_dict()
x.items()

odict_items([('net.0.weight', tensor([[ 2.9174e-01, -2.0780e-01,  6.1585e-01,  3.1392e-01,  6.4947e-02,
          6.2404e-01,  6.3532e-01,  1.3211e-01, -9.4765e-02,  6.1376e-01,
          4.5775e-01,  1.7545e-02,  4.5361e-01,  3.0078e-01,  2.3136e-01,
          6.4931e-02,  4.3706e-01,  4.0147e-01, -1.2217e-01,  4.3165e-01,
          5.0075e-01,  1.7273e-01,  2.6978e-01,  4.0746e-01,  2.2216e-01,
         -2.6892e-01,  3.3683e-01,  6.0118e-01],
        [ 2.5823e-02, -9.4434e-01,  1.9577e-01,  4.1066e-01,  4.2847e-01,
          3.5844e-01,  1.7875e-01,  2.0496e-01, -1.1071e+00,  1.3877e-01,
          4.2024e-01,  2.2419e-01, -1.0593e-01,  1.1406e-01,  7.1046e-02,
         -1.1528e+00,  3.2928e-01,  1.1434e-01, -9.2727e-02, -9.8293e-02,
          2.0220e-01,  9.2912e-02, -7.2475e-01,  7.4967e-01,  3.5431e-01,
         -1.0873e-01, -1.6552e-01,  7.2119e-02],
        [-8.8309e-02, -4.0152e-01,  5.8691e-01,  1.2121e-01, -3.3671e-01,
         -4.1393e-01, -3.5667e-01, -2.4860e-01, -3.6605e-0

In [79]:
one=(x.get('net.0.weight').numpy().tolist(), x.get('net.0.bias').numpy().tolist())

In [80]:
two=(x.get('net.2.weight').numpy().tolist(), x.get('net.2.bias').numpy().tolist())

In [81]:
WEIGHTS = [one, two]

In [92]:
for idx, (w,b) in enumerate(WEIGHTS):
  print("Layer %d:" % (idx+1))
  print("W: (%d, %d), B: (%d, %d)" % (len(w), len(w[0]), len(b), b[0]))

Layer 1:
W: (20, 28), B: (20, 0)
Layer 2:
W: (4, 20), B: (4, 0)


In [ ]:
from . import nn

WEIGHTS

def forward(vals):
    x = nn.linear(vals, WEIGHTS[0])
    nn.relu(x)
    x = nn.linear(x, WEIGHTS[1])
    nn.tanh(x)
    return x